In [16]:
import numpy as np
import pandas as pd
data_web_address ="./diabetes.csv"
column_names =['pregnancy_x',
    'plasma_con',
    'blood_pressure',
    'skin_mm',
    'insulin',
    'bmi',
    'pedigree_func',
    'age',
    'target']
feature_names = column_names[:-1]
all_data = pd.read_csv(data_web_address , names=column_names)
all_data=all_data.drop([0])
X = all_data[feature_names]
y = all_data['target']

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [21]:
from sklearn.linear_model import Perceptron
pr = Perceptron()
pr.fit(X_train_scaled, y_train)
Perceptron(alpha=0.0001, class_weight=None, eta0=1.0,
    fit_intercept=True,
     max_iter=5, n_jobs=1, penalty=None, random_state=0, shuffle=True,
     verbose=0, warm_start=False)

Perceptron(max_iter=5, n_jobs=1)

In [22]:
from sklearn.model_selection import cross_val_score,StratifiedKFold
skf = StratifiedKFold(n_splits=3)
cross_val_score(pr, X_train_scaled, y_train,
cv=skf,scoring='roc_auc').mean()

0.7130531289333276

In [23]:
from sklearn.metrics import accuracy_score, roc_auc_score
print ("Classification accuracy : ", accuracy_score(y_test,pr.predict(X_test_scaled)))
print ("ROC-AUC Score : ",roc_auc_score(y_test,pr.predict(X_test_scaled)))

Classification accuracy :  0.6753246753246753
ROC-AUC Score :  0.602020202020202


/Users/junwei1/opt/anaconda3/envs/mlfinlab/lib/python3.8/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [24]:
from sklearn.model_selection import GridSearchCV
param_dist = {'alpha': [0.1,0.01,0.001,0.0001],
    'penalty': [None, 'l2','l1','elasticnet'],
    'random_state': [7],
    'class_weight':['balanced',None],'eta0': [0.25,0.5,0.75,1.0],
    'warm_start':[True,False], 'max_iter':[50], 'tol':[1e-3]}
gs_perceptron = GridSearchCV(pr, param_dist, scoring='roc_auc',cv=skf).fit(X_train_scaled, y_train)

In [25]:
gs_perceptron.best_params_

{'alpha': 0.0001,
 'class_weight': 'balanced',
 'eta0': 0.5,
 'max_iter': 50,
 'penalty': 'elasticnet',
 'random_state': 7,
 'tol': 0.001,
 'warm_start': True}

In [26]:
gs_perceptron.best_score_

0.8025511013185685

In [27]:
best_perceptron = gs_perceptron.best_estimator_

In [30]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import BaggingClassifier
param_dist = {
    'max_samples': [0.5,1.0],
    'max_features' : [0.5,1.0],
    'oob_score' : [True, False],
    'n_estimators': [100],
    'n_jobs':[-1],
    'base_estimator__alpha': [0.001,0.002],
    'base_estimator__penalty': [None, 'l2','l1','elasticnet'], }
ensemble_estimator = BaggingClassifier(base_estimator = best_perceptron)
bag_perceptrons = GridSearchCV(ensemble_estimator,param_dist,scoring='roc_auc',cv=skf,n_jobs=-1).fit(X_train_scaled, y_train)

In [31]:
bag_perceptrons.best_score_

0.8363677745973089

In [32]:
bag_perceptrons.best_params_

{'base_estimator__alpha': 0.002,
 'base_estimator__penalty': 'elasticnet',
 'max_features': 1.0,
 'max_samples': 0.5,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': True}